### Notebook for the of Fawkner-Corbett_2021 Visium data with `squidpy`

- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Last modified date:** 22nd August 2024

#### Load packages

In [27]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import rankdata

#### Set up cells

In [28]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')

-----
anndata     0.10.5.post1
scanpy      1.9.8
-----
PIL                         10.2.0
anyio                       NA
arrow                       1.3.0
asciitree                   NA
asttokens                   NA
attr                        23.2.0
attrs                       23.2.0
babel                       2.14.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
brotli                      1.1.0
certifi                     2023.11.17
cffi                        1.16.0
charset_normalizer          3.3.2
cloudpickle                 3.0.0
comm                        0.2.1
cycler                      0.12.1
cython_runtime              NA
dask                        2024.1.1
dask_image                  2023.08.1
datashader                  0.16.0
datatree                    0.0.13
dateutil                    2.8.2
debugpy                     1.8.0
decorator                   5.1.1
defusedxml                  0.7.1
docrep                   

+ Define functions

In [29]:
def get_top_n_cell_types(row, n=1):
    sorted_types = row.sort_values(ascending=False)
    top_n = sorted_types.head(n)
    return ', '.join(top_n.index.str.replace('meanscell_abundance_w_sf_', ''))

In [30]:
def create_expanded_anndata(adata, n_cells_per_spot=10):
    # Get cell type abundances
    abundances = adata.obsm['means_cell_abundance_w_sf']
    
    # Create a list to store new observations
    new_obs = []
    new_obs_names = []
    
    for idx, row in abundances.iterrows():
        # Normalize abundances to sum to 1
        normalized_abundances = row / row.sum()
        
        # Calculate the number of cells for each type
        cell_counts = np.random.multinomial(n_cells_per_spot, normalized_abundances)
        
        for cell_type, count in zip(row.index, cell_counts):
            for i in range(count):
                new_obs.append({
                    'original_barcode': idx,
                    'cell_type': cell_type.replace('meanscell_abundance_w_sf_', '')
                })
                new_obs_names.append(f"{idx}-{str(i+1).zfill(2)}")
    
    # Create a new AnnData object
    new_adata = sc.AnnData(
        X=np.zeros((len(new_obs), adata.n_vars)),
        obs=pd.DataFrame(new_obs, index=new_obs_names)
    )
    
    # Add spatial coordinates
    original_coords = adata.obsm['spatial']
    new_coords = np.zeros((len(new_obs), original_coords.shape[1]))
    
    for i, obs in enumerate(new_obs):
        orig_idx = adata.obs_names.get_loc(obs['original_barcode'])
        new_coords[i] = original_coords[orig_idx]
    
    new_adata.obsm['spatial'] = new_coords
    
    return new_adata

## Adult colon samples

#### Sample A1

In [31]:
dir_path = '/../../../gut_project/Processed_data/Gut_data/Visium_fawkner_corbett/cell2location_results'
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_3464locations_48factors/anndata/sp.h5ad'

In [32]:
sample = 'A1'

In [33]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [34]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [35]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4122.48/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [36]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment_grid.png", bbox_inches="tight")
    plt.close()

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [37]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="Stem cells", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_Stem.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.52/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


+ Option 2 

In [38]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [39]:
cells_per_spot = new_adata.obs['original_barcode'].value_counts()
print("Number of cells per spot:")
print(cells_per_spot.value_counts())

Number of cells per spot:
count
10    3464
Name: count, dtype: int64


In [40]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [41]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 825.19/s]

Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:01)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [42]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A2

In [43]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_2810locations_48factors/anndata/sp.h5ad'

In [44]:
sample = 'A2'

In [45]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [46]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [47]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4674.75/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [48]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [49]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="Stem cells", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_Stem.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.47/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [50]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [51]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [52]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 670.84/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:01)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [53]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A3

In [54]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_2782locations_95factors/anndata/sp.h5ad'

In [55]:
sample = 'A3'

In [56]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [57]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [58]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3038.44/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [59]:
adata.obs['top_cell_types'].value_counts()

top_cell_types
Stromal 3 (C7+)                1236
SMC (PART1/CAPN3+)              986
Colonocyte                      327
Stromal 2 (NPY+)                130
Contractile pericyte (PLN+)      16
SMC (PLPP2+)                     15
LEC4 (STAB2+)                    11
Fetal venous EC                  11
cycling stromal                  10
Stromal 1 (ADAMDEC1+)             8
myofibroblast (RSPO2+)            7
Glia 1 (DHH+)                     4
SELL+ CD8 T                       4
Branch B2 (eMN)                   3
Mesoderm 2 (ZEB2+)                2
FXYD3+_CKB+_SC                    2
Enterocyte                        2
Activated T                       2
BEST4+ epithelial                 1
RBC                               1
Branch A4 (IN)                    1
Branch A2 (IPAN/IN)               1
TA                                1
Branch A1 (iMN)                   1
Name: count, dtype: int64

In [60]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()    

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [61]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_FXYD3+_CKB+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.20/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [62]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [63]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [64]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 795.13/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:01)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [65]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(17, 17))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(17, 17))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()   

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A4

In [66]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_2265locations_95factors/anndata/sp.h5ad'

In [67]:
sample = 'A4'

In [68]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [69]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [70]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4912.39/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [71]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()    

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [72]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_FXYD3+_CKB+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.17/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [73]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [74]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [75]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 953.99/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:01)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [76]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(17, 17))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(17, 17))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()   

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A6

In [77]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_1677locations_100factors/anndata/sp.h5ad'

In [78]:
sample = 'A6'

In [79]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [80]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [81]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4895.68/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [82]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="RPS10+_RPS17+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_RPS10+_RPS17+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.79/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [83]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_FXYD3+_CKB+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.95/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [84]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="MTRNR2L12+ASS1+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_MTRNR2L12+ASS1+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.98/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [85]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()    

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [86]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [87]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [88]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1188.30/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:00)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [89]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(17, 17))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(17, 17))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()   

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A7

In [90]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_1691locations_100factors/anndata/sp.h5ad'

In [91]:
sample = 'A7'

In [92]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [93]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [94]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5156.19/s]


Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [95]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()    

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [96]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_FXYD3+_CKB+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.09/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [97]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="RPS10+_RPS17+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_RPS10+_RPS17+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.51/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [98]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="MTRNR2L12+ASS1+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence_MTRNR2L12+ASS1+_SC.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.66/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [99]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [100]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [101]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1129.73/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:00)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [102]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(17, 17))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(17, 17))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()   

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A8

In [103]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_2600locations_95factors/anndata/sp.h5ad'

In [104]:
sample = 'A8'

In [105]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [106]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [107]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4352.60/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [108]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()    

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [109]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.83/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [110]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [111]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [112]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 790.71/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:01)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [113]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(17, 17))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(17, 17))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()   

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))


### A9

In [114]:
adata_path = 'CoLocatedComb/CoLocatedGroupsSklearnNMF_2255locations_95factors/anndata/sp.h5ad'

In [115]:
sample = 'A9'

In [116]:
adata = sc.read_h5ad(f'{dir_path}/{sample}/{adata_path}')

In [117]:
adata.obs['top_cell_types'] = adata.obsm['means_cell_abundance_w_sf'].apply(get_top_n_cell_types, axis=1)
adata.obs['top_cell_types'] = adata.obs['top_cell_types'].astype('category')

In [118]:
adata.obs['top_cell_types'].value_counts()

top_cell_types
Stromal 3 (C7+)                1044
Colonocyte                      878
SMC (PART1/CAPN3+)              272
SMC (PLPP2+)                     15
Enterocyte                       14
Stromal 2 (CH25H+)                7
SELL+ CD8 T                       5
cycling stromal                   4
Stromal 2 (NPY+)                  4
Contractile pericyte (PLN+)       3
FXYD3+_CKB+_SC                    2
Branch A2 (IPAN/IN)               1
Mesoderm 2 (ZEB2+)                1
Branch A3 (IPAN/IN)               1
LEC4 (STAB2+)                     1
Glia 1 (DHH+)                     1
Branch B2 (eMN)                   1
myofibroblast (RSPO2+)            1
Name: count, dtype: int64

In [119]:
sq.gr.spatial_neighbors(adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(adata, cluster_key="top_cell_types")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|██████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4773.62/s]

Adding `adata.uns['top_cell_types_nhood_enrichment']`
Finish (0:00:00)



/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [120]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.nhood_enrichment(adata, cluster_key="top_cell_types", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment.png", bbox_inches="tight")
    plt.close()    

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]


In [121]:
sq.gr.co_occurrence(adata, cluster_key="top_cell_types")
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(4.5, 5))
    sq.pl.co_occurrence(adata, cluster_key="top_cell_types", clusters="FXYD3+_CKB+_SC", figsize=(10, 10))
    plt.savefig( f"{dir_path}/{sample}/{sample}_co_occurrence.png", bbox_inches="tight")
    plt.close() 

Calculating co-occurrence probabilities for `50` intervals `1` split combinations using `1` core(s)


100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.28/s]

Adding `adata.uns['top_cell_types_co_occurrence']`
Finish (0:00:00)
ERROR: Unable to fetch palette, reason: 'top_cell_types_colors'. Using `None`.


In [122]:
new_adata = create_expanded_anndata(adata)

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [123]:
new_adata.obs["cell_type"] = new_adata.obs["cell_type"].astype("category")

In [124]:
sq.gr.spatial_neighbors(new_adata, n_rings=2, coord_type="grid", n_neighs=6)
sq.gr.nhood_enrichment(new_adata, cluster_key="cell_type")

Creating graph using `grid` coordinates and `None` transform and `1` libraries.
Adding `adata.obsp['spatial_connectivities']`
       `adata.obsp['spatial_distances']`
       `adata.uns['spatial_neighbors']`
Finish (0:00:00)
Calculating neighborhood enrichment using `1` core(s)


100%|███████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 953.79/s]


Adding `adata.uns['cell_type_nhood_enrichment']`
Finish (0:00:01)


/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/gr/_nhood.py:188: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [125]:
with plt.rc_context():
    sc.set_figure_params(dpi=300, figsize=(17, 17))
    sq.pl.nhood_enrichment(new_adata, cluster_key="cell_type", figsize=(17, 17))
    plt.savefig( f"{dir_path}/{sample}/{sample}_nhood_enrichment2.png", bbox_inches="tight")
    plt.close()   

/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:556: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_labels = adata.obs[key][row_order]
/home/amaguza/miniforge3/envs/squidpy_env/lib/python3.9/site-packages/squidpy/pl/_utils.py:564: RuntimeWarning: All-NaN axis encountered
  norm = mpl.colors.Normalize(vmin=kwargs.pop("vmin", np.nanmin(data)), vmax=kwargs.pop("vmax", np.nanmax(data)))
